## Data Understanding

In [1]:
# Import Libraries
import pandas as pd
import numpy as np

In [2]:
# Read the data into a dataframe
df_accidents = pd.read_json('crash_data.json')

In [3]:
class DataUnderstanding:
    """This class gives the general quick overview of a Dataframe"""
    
    def __init__(self, dataframe):
        self.df = dataframe
        
    def print_shape(self, title="Record"):
        """This function gives us the shape of the data"""
        print(title.upper())
        print("-"*len(title))
        print("There are", self.df.shape[0], "rows")
        print("There are", self.df.shape[1], "columns")
        print("\n")
        
    def information(self, title="Info"):
        """This function gives us the descriptive information of the data"""
        print(title.upper())
        print("-"*len(title))
        print(self.df.info())
        print("\n")
        
    def duplicate_values(self, title="Duplicate Values"):
        """This function gives us the number of duplicates in the data"""
        print(title.upper())
        print("-"*len(title))
        try:
            print(f"There are {self.df.duplicated().sum()} duplicated values in this dataset")
        except:
            print("There was a Type Error: One column has a 'dict' values")
        print("\n")
        
    def null_values(self, title="Null Values"):
        """This function gives us the number of null values in the data"""
        print(title.upper())
        print("-"*len(title))
        isnulls = self.df.isnull().sum()
        print(f"There are {isnulls.sum()} null values in this dataset")
        print("\n")
        
        print(f"There are {len(isnulls[isnulls > 0])} columns with null values while {len(isnulls[isnulls == 0])}"\
             +" do not have null values")
        print("\n")
        
        print("Null Values per column")
        print(isnulls[isnulls > 0])
        print("\n")        
        
    def unique_values(self, title="Unique Values"):
        """This function gives us the unique and nunique values in the data"""
        print(title.upper())
        print("-"*len(title))
        for col in self.df.columns:
            print(col)
            print("-"*len(col))
            try:
                print(f"Number of unique values: {self.df[col].nunique()}")
            except:
                print("Error - This column has a type error")
            print("\n")
        
        
    def run_all(self):
        self.print_shape()
        self.information()
        self.duplicate_values()
        self.null_values()
        self.unique_values()


In [4]:
# Implement the class on the dataframe
data = DataUnderstanding(df_accidents)
data.run_all()

RECORD
------
There are 1000 rows
There are 44 columns


INFO
----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 44 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   report_number                 1000 non-null   object        
 1   local_case_number             1000 non-null   int64         
 2   agency_name                   1000 non-null   object        
 3   acrs_report_type              1000 non-null   object        
 4   crash_date_time               1000 non-null   datetime64[ns]
 5   number_of_lanes               1000 non-null   object        
 6   distance_unit                 1000 non-null   object        
 7   off_road_description          133 non-null    object        
 8   at_fault                      1000 non-null   object        
 9   collision_type                1000 non-null   object        
 10  weather                       

## Data Cleaning

### 1. Standardising Column Names and Values

#### 1A. Columns to retain

In [5]:
# Columns to keep
accidents_columns = [
       'agency_name', 'acrs_report_type',
       'crash_date_time', 'number_of_lanes', 'at_fault', 'collision_type', 'weather',
       'light', 'driver_substance_abuse', 'first_harmful_event', 'latitude',
       'longitude', 'hit_run', 'road_grade', 'surface_condition',
       'traffic_control', 'junction', 'intersection_type', 'road_alignment',
       'road_condition', 'road_division', 'route_type', 'road_name',
       ]

In [6]:
# Create a new dataframe with the specified columns
df = df_accidents[accidents_columns].copy()

#### 1B. Drop Some Records

In [7]:
# Drop rows without critical data i.e where the offroad description was there
df = df[(df['road_grade'].isna() != True ) & (df['surface_condition'].isna() != True ) & 
        (df['traffic_control'].isna() != True )].copy()

### 2. Too many values 

#### Helper Functions

In [9]:
def clean_driver_substance_abuse(df, column_name='driver_substance_abuse'):
    """
    Cleans the 'Driver Substance Abuse' column by mapping unique values to key phrases.
    """
    mapping = {
        # Alcohol
        'ALCOHOL PRESENT': 'Alcohol',
        'ALCOHOL CONTRIBUTED': 'Alcohol',
        'ALCOHOL PRESENT, N/A': 'Alcohol',
        'ALCOHOL CONTRIBUTED, N/A': 'Alcohol',
        'ALCOHOL PRESENT, NONE DETECTED': 'Alcohol',
        'ALCOHOL CONTRIBUTED, NONE DETECTED': 'Alcohol',
        'ALCOHOL PRESENT, N/A, NONE DETECTED': 'Alcohol',
        'ALCOHOL PRESENT, UNKNOWN': 'Alcohol',
        'ALCOHOL CONTRIBUTED, UNKNOWN': 'Alcohol',
        'ALCOHOL PRESENT, NONE DETECTED, UNKNOWN': 'Alcohol',
        'ALCOHOL CONTRIBUTED, NONE DETECTED, UNKNOWN': 'Alcohol',
        'ALCOHOL PRESENT, N/A, UNKNOWN': 'Alcohol',
        'ALCOHOL CONTRIBUTED, N/A, NONE DETECTED': 'Alcohol',
        'ALCOHOL PRESENT, ILLEGAL DRUG PRESENT': 'Alcohol, Illegal Drugs',
        'Suspect of Alcohol Use, Not Suspect of Drug Use': 'Alcohol',
        'Suspect of Alcohol Use, Unknown': 'Alcohol',
        'Suspect of Alcohol Use, Suspect of Drug Use': 'Alcohol, Illegal Drugs',
        'Suspect of Alcohol Use, Suspect of Drug Use, Unknown, Not Suspect of Drug Use': 'Alcohol, Illegal Drugs',
        'Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Unknown': 'Alcohol',
        'Suspect of Alcohol Use, Not Suspect of Drug Use, Suspect of Alcohol Use, Not Suspect of Drug Use': 'Alcohol',
        'Suspect of Alcohol Use, Unknown, Unknown, Unknown': 'Alcohol',
        
        # Drugs
        'ILLEGAL DRUG CONTRIBUTED, NONE DETECTED': 'Illegal Drugs',
        'ILLEGAL DRUG PRESENT, NONE DETECTED': 'Illegal Drugs',
        'ILLEGAL DRUG PRESENT': 'Illegal Drugs',
        'ILLEGAL DRUG CONTRIBUTED': 'Illegal Drugs',
        'ILLEGAL DRUG PRESENT, N/A': 'Illegal Drugs',
        'ILLEGAL DRUG CONTRIBUTED, N/A': 'Illegal Drugs',
        'ILLEGAL DRUG PRESENT, N/A, NONE DETECTED': 'Illegal Drugs',
        'ILLEGAL DRUG PRESENT, UNKNOWN': 'Illegal Drugs',
        'Not Suspect of Alcohol Use, Suspect of Drug Use': 'Illegal Drugs',
        'Unknown, Not Suspect of Drug Use': 'Illegal Drugs',
        'Not Suspect of Alcohol Use, Suspect of Drug Use, Unknown, Unknown': 'Illegal Drugs',
        'Not Suspect of Alcohol Use, Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use': 'Illegal Drugs',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Suspect of Drug Use': 'Illegal Drugs',
        'Suspect of Alcohol Use, Suspect of Drug Use, Unknown, Unknown': 'Alcohol, Illegal Drugs',
        'Unknown, Suspect of Drug Use': 'Illegal Drugs',

        # Medication
        'MEDICATION PRESENT, NONE DETECTED': 'Medication',
        'MEDICATION PRESENT': 'Medication',
        'MEDICATION CONTRIBUTED, NONE DETECTED, UNKNOWN': 'Medication',
        'MEDICATION CONTRIBUTED, NONE DETECTED': 'Medication',
        'MEDICATION CONTRIBUTED, UNKNOWN': 'Medication',
        'MEDICATION PRESENT, N/A': 'Medication',
        'MEDICATION CONTRIBUTED': 'Medication',
        'MEDICATION CONTRIBUTED, N/A': 'Medication',
        'MEDICATION PRESENT, NONE DETECTED, UNKNOWN': 'Medication',
        'MEDICATION PRESENT, N/A, UNKNOWN': 'Medication',

        # Combined drugs
        'COMBINATION CONTRIBUTED, N/A': 'Combined Substance',
        'COMBINED SUBSTANCE PRESENT, NONE DETECTED': 'Combined Substance',
        'COMBINED SUBSTANCE PRESENT': 'Combined Substance',
        'COMBINED SUBSTANCE PRESENT, N/A': 'Combined Substance',
        'COMBINATION CONTRIBUTED, NONE DETECTED': 'Combined Substance',
        'COMBINED SUBSTANCE PRESENT, UNKNOWN': 'Combined Substance',
        'COMBINED SUBSTANCE PRESENT, N/A, NONE DETECTED': 'Combined Substance',
        'COMBINATION CONTRIBUTED': 'Combined Substance',

        # No drugs or Alcohol
        'NONE DETECTED': 'None Detected',
        'N/A, NONE DETECTED': 'None Detected',
        'NONE DETECTED, UNKNOWN': 'None Detected',
        'N/A, NONE DETECTED, UNKNOWN': 'None Detected',

        # Unknown
        'UNKNOWN': 'Unknown',
        'N/A, UNKNOWN': 'Unknown',
         np.nan: 'Unknown',
        'Unknown, Unknown': 'Unknown',
        'Unknown, Unknown, Unknown, Unknown': 'Unknown',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Unknown': 'Unknown',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Unknown, Unknown, Unknown': 'Unknown',
        'Unknown, Unknown, Unknown, Unknown, Unknown, Unknown': 'Unknown',
        'Unknown, Unknown, Unknown, Unknown, Unknown, Unknown, Unknown, Unknown': 'Unknown',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Unknown, Unknown, Unknown, Unknown, Unknown': 'Unknown',
        'Unknown, Not Suspect of Drug Use, Unknown, Unknown': 'Unknown',
        'Suspect of Alcohol Use, Unknown, Unknown, Unknown': 'Unknown',
        'Not Suspect of Alcohol Use, Unknown, Unknown, Unknown': 'Unknown',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Unknown': 'Unknown',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Alcohol Use, Unknown, Unknown': 'Unknown',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Alcohol Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Unknown': 'Unknown',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Alcohol Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Unknown': 'Unknown',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Alcohol Use, Not Suspect of Alcohol Use, Not Suspect of Alcohol Use, Not Suspect of Alcohol Use, Not Suspect of Alcohol Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use': 'Unknown',
        'Not Suspect of Alcohol Use, Unknown, Not Suspect of Alcohol Use, Unknown, Not Suspect of Alcohol Use, Unknown': 'Unknown',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Not Suspect of Drug Use': 'Unknown',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Unknown': 'Unknown',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Unknown': 'Unknown',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Not Suspect of Drug Use': 'Unknown',
        'Unknown, Not Suspect of Drug Use, Unknown, Unknown': 'Unknown',

        # Not Suspected
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use': 'Not Suspected',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use': 'Not Suspected',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use': 'Not Suspected',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use': 'Not Suspected',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use': 'Not Suspected',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use': 'Not Suspected',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Suspect of Alcohol Use, Not Suspect of Drug Use': 'Not Suspected',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use': 'Not Suspected',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Suspect of Alcohol Use, Not Suspect of Drug Use': 'Not Suspected',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Suspect of Alcohol Use, Suspect of Drug Use': 'Not Suspected',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Suspect of Alcohol Use, Unknown': 'Not Suspected',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Suspect of Alcohol Use, Suspect of Drug Use': 'Not Suspected',
        'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Suspect of Alcohol Use, Suspect of Drug Use': 'Not Suspected',
        'Not Suspect of Alcohol Use, Unknown': 'Not Suspected',
        'Not Suspect of Alcohol Use, Unknown, Not Suspect of Alcohol Use, Not Suspect of Drug Use': 'Not Suspected',

        # Others
        'OTHER': 'Other',
        'N/A, OTHER': 'Other',
        'NONE DETECTED, OTHER': 'Other',
    }

    # The mapping function - any null items are filled using 'Other'
    df.loc[:, column_name] = df[column_name].map(mapping).fillna('Other')

    print(df[column_name].unique())
    
def clean_traffic_control(df, column_name='Traffic Control'):
    """
    Cleans the Traffic Control column by mapping unique values to key phrases.
    """
    # No/Unknown/Miscellaneous Control
    mapping = {
    'No Controls': 'No/Unknown/Miscellaneous Control',
    'Unknown': 'No/Unknown/Miscellaneous Control',
    'Other': 'No/Unknown/Miscellaneous Control',
    'Other Signal': 'No/Unknown/Miscellaneous Control',
    'Other Pavement Marking (Excluding Edgelines, Centerlines, Or Lane Lines)': 'No/Unknown/Miscellaneous Control',
    'School Zone': 'No/Unknown/Miscellaneous Control', # If it's a general area description

    # Signs
    'Stop Sign': 'Signs',
    'Yield Sign': 'Signs',
    'Warning Sign': 'Signs',
    'School Zone Sign Device': 'Signs',
    'School Zone Sign': 'Signs',
    'Pedestrian Crossing Sign': 'Signs',
    'Bicycle Crossing Sign': 'Signs',
    'Intersection Ahead Warning Sign': 'Signs',
    'Curve Ahead Warning Sign': 'Signs',
    'Reduce Speed Ahead Warning Sign': 'Signs',
    'Other Warning Sign': 'Signs',

    # Traffic Signals
    'Traffic Signal': 'Traffic Signals',
    'Flashing Traffic Signal': 'Traffic Signals',
    'Traffic Control Signal': 'Traffic Signals',
    'Flashing Traffic Control Signal': 'Traffic Signals',
    'Ramp Meter Signal': 'Traffic Signals',
    'Lane Use Control Signal': 'Traffic Signals',

    # Railroad Crossing Devices
    'Railway Crossing Device': 'Railroad Crossing Devices',
    'Flashing Railroad Crossing Signal (May Include Gates)': 'Railroad Crossing Devices',

    # Human/Pedestrian Control
    'Person': 'Human/Pedestrian Control',
    'Person (Including Flagger, Law Enforcement, Crossing Guard, Etc.': 'Human/Pedestrian Control',
    'Pedestrian Crossing': 'Human/Pedestrian Control'
    }
        
    # The mapping function - any null items are filled using 'Other'
    df.loc[:, column_name] = df[column_name].map(mapping).fillna('Other') 

    print(df[column_name].unique())

#### 2A. Clean The Driver Substance Abuse Col

In [10]:
# Apply cleaning function
clean_driver_substance_abuse(df)

['Not Suspected' 'Unknown' 'Alcohol' 'Other' 'Alcohol, Illegal Drugs'
 'Illegal Drugs']


#### 2B. Normalise all values in the columns

In [11]:
# Only the categorical columns
categorical_cols = list(df.select_dtypes('O').columns)
categorical_cols

# # Standardise values in the categorical columns
for col in categorical_cols:
    df[col] = df[col].apply(lambda x: x.title() if type(x) == str else x)

#### 2C. Clean the Traffic Control Column


In [12]:
# Apply the cleaning on the traffic control column
clean_traffic_control(df, 'traffic_control')

['Signs' 'Other' 'No/Unknown/Miscellaneous Control' 'Traffic Signals'
 'Human/Pedestrian Control']


### 3. Replace NANs with specified value

In [13]:
def replace_values(replacing_dict:dict):
    """
    This function uses the column (key) plus the replacement value (value) from the 
    dictionary provided to eradicate nulls from the data
    """
    for column, value in replacing_dict.items():
        df.loc[:, column] = df[column].apply(lambda x: value if type(x) != str else x)

In [14]:
# Dictionary with the column and replacement value for any nulls
replace_dictionary = {
    'intersection_type': "Non-Intersection", 
    "hit_run": "Unknown", 
    "road_name": 'Undisclosed'
}

# Apply the created function
replace_values(replacing_dict=replace_dictionary)

### 4. Fill Null Values with Most common Value

In [15]:
# Columns with null values that should be filled with the mode
fill_null_columns = ['junction', 'route_type']

for col in fill_null_columns:
    col_mode = df[col].mode()[0] # Get the mode of the column
    df.loc[:, col] = df[col].fillna(col_mode) # Fill with the mode
    
# Check
for col in fill_null_columns:
    print(col, "missing values", df[col].isna().sum())

junction missing values 0
route_type missing values 0


### 5. Final Check

In [16]:
# Save Cleaned Dataset
df_cleaned = df.to_csv('cleaned_data.csv')

cleaned_df = DataUnderstanding(df)
cleaned_df.run_all()

RECORD
------
There are 862 rows
There are 23 columns


INFO
----
<class 'pandas.core.frame.DataFrame'>
Int64Index: 862 entries, 1 to 999
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   agency_name             862 non-null    object        
 1   acrs_report_type        862 non-null    object        
 2   crash_date_time         862 non-null    datetime64[ns]
 3   number_of_lanes         862 non-null    object        
 4   at_fault                862 non-null    object        
 5   collision_type          862 non-null    object        
 6   weather                 862 non-null    object        
 7   light                   862 non-null    object        
 8   driver_substance_abuse  862 non-null    object        
 9   first_harmful_event     862 non-null    object        
 10  latitude                862 non-null    float64       
 11  longitude               862 non-null    floa

## Data Warehousing

This section focuses on creating fact tables and dimension tables. This follows the star schema model. 

The star schema is used because it is:
- Reusable
- Optimises efficiency
- Scalable
- Enhances digital integrity

In [18]:
def create_mappings(df, cat_cols):
    """
    This function creates a mapping of the datasets categorical columns' unique values.
    It returns a list full of the mappings per categorical column
    """

    # Stores the lists created
    mapping_dictionary  = []

    # For each categorical column create a mapping of the values and assign numbers to it
    for col in categorical_columns:
        mapping_dictionary.append(
            {
                col: {value: key for key, value in dict(enumerate(df[col].unique(), 1)).items()}
            }
        )
        
    return mapping_dictionary 

def retrieve_map(col):
    """This function retrieves one map dictionary for a specified column"""
    for map in df_mappings:
        for key, value in map.items():
            if key == col:
                return value
            
def replace_mapped_values(column_to_be_mapped, mapping_list, df=df):
    """
    This function maps the each columns' value to the map provided in the mapping variable.
    This will turn the categorical values into numerical values
    """
    # Retrieves one map for the column chosen
    column_specific_map = retrieve_map(column_to_be_mapped)
    
    # Makes the actual mapping
    df.loc[:, column_to_be_mapped] = df[column_to_be_mapped].map(column_specific_map)

### <i>Fact Table<i>

In [19]:
# Create a seperate copy of the cleaned data
mapped_df = df.copy()

In [20]:
# Find the categorical columns
categorical_columns = list(mapped_df.select_dtypes('O'))

# Apply the create mappings function
df_mappings = create_mappings(mapped_df, categorical_columns)

In [21]:
# Apply mapping to the new dataframe
for col in categorical_columns:
    replace_mapped_values(column_to_be_mapped=col, mapping_list=df_mappings, df=mapped_df)

<ipython-input-18-06f36cbd61de>:36: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, column_to_be_mapped] = df[column_to_be_mapped].map(column_specific_map)
<ipython-input-18-06f36cbd61de>:36: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, column_to_be_mapped] = df[column_to_be_mapped].map(column_specific_map)
<ipython-input-18-06f36cbd61de>:36: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use eithe

In [22]:
# Check
mapped_df.iloc[:5, 9:].head()

,first_harmful_event,latitude,longitude,hit_run,road_grade,surface_condition,traffic_control,junction,intersection_type,road_alignment,road_condition,road_division,route_type,road_name
1,1,39.004033,-77.040995,1,1,1,1,1,1,1,1,1,1,1
3,1,39.019026,-77.015578,1,2,1,2,1,1,1,1,1,1,2
4,2,39.122000,-77.115599,1,2,1,3,2,2,1,1,2,1,3
6,1,39.001233,-77.123193,1,1,2,2,1,3,1,1,3,1,4
7,1,39.044978,-77.111125,1,3,2,4,2,2,2,1,4,1,5


In [23]:
# Save Fact Table as csv
mapped_df.reset_index(inplace=True, drop=True)
mapped_df.index = mapped_df.index + 1
mapped_df.head()
crash_data_fact_table = mapped_df.to_csv('crash_data_fact_table.csv')

### <i>Dimension Tables<i>

In [25]:
# Empty Dictionary for holding the dataframes
dimensions_dictionary = {}

# Each categorical column will create a dictionary of unique values with an index
for col in categorical_columns:
    col_renamed = col.replace(' ', '_') # Format the col value
    columns = [col_renamed+'_ID', col_renamed+'_Status'] # Columns of the DataFrame
    col_mapped_dictionary = dict(enumerate(df[col].unique(), 1)) # Dictionary of index and unique values
    dataframe = pd.DataFrame(col_mapped_dictionary.items(), columns=columns) # Create dataframe
    dimensions_dictionary[col_renamed] = dataframe # Save dataframe in the dimensions_dictionary

# Check
dimensions_dictionary['agency_name']

,agency_name_ID,agency_name_Status
0,1,Montgomery
1,2,Takoma
2,3,Rockville
3,4,Gaithersburg
4,5,Mcpark


### Postgres Quick Query Creation

In [ ]:
# This query will create a text that will create the fact table using PostgreSQL syntax
for col in mapped_df.columns:
    if col not in ['crash_date_time', 'latitude', 'longitude']:
        table_name = 'dim_' + col
        table_name = dimensions_dictionary[col].to_csv(f"{table_name}.csv")
        text = \
        f""" 
        DROP TABLE IF EXISTS {col};
        CREATE TABLE {col}(
            id SERIAL PRIMARY KEY,
            {dimensions_dictionary[col].columns[0]} INT,
            {dimensions_dictionary[col].columns[1]} VARCHAR(255)            
        );
        """
        print(text)
        print()
   
# This query will create a text that will upload the dimension data csvs using PostgreSQL syntax - use it in psql tool
for col in mapped_df.columns:
    if col not in ['crash_date_time', 'latitude', 'longitude']:
        table_name = 'dim_' + col
        text = \
        f"""\COPY {col} FROM 'C:\\Users\\PROBOOK 6460\\Documents\\EU BUSINESS SCHOOL\\Business Intelligence\\Final Project\\{table_name}.csv' WITH CSV HEADER;
        """
        print(text)
        print()